In [1]:
from sklearn.datasets import load_boston
from sklearn.tree import DecisionTreeRegressor
import numpy as np
import matplotlib.pyplot as pl
from pandas import DataFrame
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [2]:
class Empty_exception(BaseException):
    pass

def penalty(sample, answers, quan, tag):
    left, right = [], []
    for elem, ind in zip(sample, range(len(sample))):
        if elem[tag] <= quan:
            left.append(ind)
        else:
            right.append(ind)
    
    if len(left) == 0 or len(right) == 0:
        raise Empty_exception()
    
    return (len(left) * MSE(answers[left]) + len(right) * MSE(answers[right])) / (len(left) + len(right))
    
def MSE(arr):
    return np.mean((arr - np.mean(arr))**2)   

In [3]:
class Node:
    
    def __init__(self, sample, answers, quantiles, max_lvl_below, min_elem_leaf):
        self.best_params = [None, None, None]
        self.left_son = None
        self.right_son = None
        if max_lvl_below > 0 and len(sample) >= min_elem_leaf:
            for tag in range(len(quantiles)):
                for quan in quantiles[tag]:
                    try:
                        current_penalty = penalty(sample, answers, quan, tag)
                    except(Empty_exception):
                        continue
                    if self.best_params[0] == None or self.best_params[0] > current_penalty:
                        self.best_params = [current_penalty, tag, quan]
            left, right = [], []
            for elem, ind in zip(sample, range(len(sample))):
                if elem[self.best_params[1]] <= self.best_params[2]:
                    left.append(ind)
                else:
                    right.append(ind)
            
            self.left_son = Node(sample[left], answers[left], quantiles, max_lvl_below - 1, min_elem_leaf)
            self.right_son = Node(sample[right], answers[right], quantiles, max_lvl_below - 1, min_elem_leaf)
        else:
            self.prediction = np.mean(answers)

    def predict(self, elem):
        if self.left_son == None:
            return self.prediction
        else:
            if elem[self.best_params[1]] <= self.best_params[2]:
                return self.left_son.predict(elem)
            else:
                return self.right_son.predict(elem)

    
class decision_tree:
    
    def __init__(self, max_h, min_elem_in_leaf=2):
        self.max_height = max_h
        self.min_elem_leaf = min_elem_in_leaf
    
    def fit(self, sample, answers):
        quantiles = [[sorted(tag_sample)[int(j * len(tag_sample))] 
                           for j in np.linspace(0.1, 0.9, 5)] for tag_sample in sample.T]
        self.head = Node(sample, answers, quantiles, self.max_height, self.min_elem_leaf)
    
    def predict(self, sample):
        return [self.head.predict(i) for i in sample]

In [4]:
boston = load_boston()

train_data = boston['data'][:380]
train_target = boston['target'][:380]
test_data = boston['data'][380:]
test_target = boston['target'][380:]

In [5]:
trees = [decision_tree(3, 2), DecisionTreeRegressor(max_depth=5)]
predictions = []

for tree in trees:
    tree.fit(train_data, train_target)
    predictions.append(tree.predict(test_data))

Сравним ошибку моего дерева с ошибкой sklearn'овского дерева и классификатора, который просто выдает среднее значение на обучающей выборке.

In [7]:
print("my_tree={}, sklearn_tree={}, constant={}".format(mean_squared_error(predictions[0], test_target), 
      mean_squared_error(predictions[1], test_target),
      mean_squared_error([np.mean(train_target) for i in predictions[0]], test_target)))

my_tree=41.29970359680129, sklearn_tree=38.517708539843554, constant=129.99148242096467


Видим, что мое дерево неплохо работает)